In [ ]:
import os

def list_files(startpath):
    # Folders to skip to keep output readable
    skip_dirs = {'.git', '.dvc', 'venv311', '__pycache__', '.ipynb_checkpoints'}
    
    for root, dirs, files in os.walk(startpath):
        # Filter out skipped directories
        dirs[:] = [d for d in dirs if d not in skip_dirs]
        
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print(f'{indent}{os.path.basename(root)}/')
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print(f'{subindent}{f}')

if __name__ == "__main__":
    print(f"Current Structure for: {os.getcwd()}\n")
    list_files('..')

In [ ]:
print("hi")

In [ ]:
#basic path test 
import sys
import os
from pathlib import Path
import pandas as pd

# 1. Fix the path logic so the Notebook can see the 'utils' folder
# This assumes your notebook is in a subfolder. If it's in the root, use "."
sys.path.append(os.path.abspath("..")) 

try:
    from utils.paths import (
        BASE_DIR, 
        RAW_SERPAPI_DIR, 
        PROCESSED_SERPAPI_DIR, 
        LOG_DIR, 
        JOBS_CSV_PATH, 
        LOCATIONS_YAML_PATH, 
        PARAMS_PATH
    )

    # 2. Define the paths to check
    paths_to_check = {
        "Project Root (BASE_DIR)": BASE_DIR,
        "Raw Data Folder": RAW_SERPAPI_DIR,
        "Processed Data Folder": PROCESSED_SERPAPI_DIR,
        "Logs Folder": LOG_DIR,
        "Jobs CSV File": JOBS_CSV_PATH,
        "Locations YAML File": LOCATIONS_YAML_PATH,
        "Global paramas file ": PARAMS_PATH
    }

    # 3. Perform Validation
    results = []
    for name, path in paths_to_check.items():
        results.append({
            "Component": name,
            "Exists": "✅ Yes" if path.exists() else "❌ NO",
            "Type": "Directory" if path.is_dir() else "File" if path.is_file() else "Missing",
            "Absolute Path": str(path.resolve())
        })

    # 4. Display as a nice Table
    df_results = pd.DataFrame(results)
    display(df_results)

    # 5. Summary Warning
    missing_count = sum(1 for r in results if "❌" in r["Exists"])
    if missing_count > 0:
        print(f"\n⚠️ WARNING: {missing_count} path(s) are invalid. Check your BASE_DIR in utils/paths.py!")
    else:
        print("\n🚀 ALL SYSTEM PATHS ARE VALID!")

except ModuleNotFoundError:
    print("❌ ERROR: Could not find 'utils' folder. Ensure you are running the notebook from the correct directory.")

In [ ]:
import sys
import os
from pathlib import Path
import pandas as pd

# 1. Fix the path logic so the Notebook can see the 'utils' folder
# This assumes your notebook is in a subfolder. If it's in the root, use "."
sys.path.append(os.path.abspath("..")) 


from utils.paths import (
        BASE_DIR, 
        RAW_SERPAPI_DIR, 
        PROCESSED_SERPAPI_DIR, 
        LOG_DIR, 
        JOBS_CSV_PATH, 
        LOCATIONS_YAML_PATH
)

print(BASE_DIR)


In [ ]:
import pandas as pd
from utils.paths import get_processed_data_path
from utils.dates import current_run_month

# 1. Load your latest processed data
csv_path = get_processed_data_path(current_run_month())
df = pd.read_csv(csv_path)

# 2. Define the placeholders we used in process_data.py
placeholders = ["Unknown", "Not mentioned", "None"]

# 3. Create the report
report_data = []

for col in df.columns:
    # Standard NaNs
    nan_count = df[col].isnull().sum()
    
    # Placeholder counts (Unknown, etc.)
    placeholder_count = df[col].astype(str).isin(placeholders).sum()
    
    total_missing = nan_count + placeholder_count
    
    report_data.append({
        "Column": col,
        "Actual NaN": nan_count,
        "Placeholders": placeholder_count,
        "Total Missing": total_missing,
        "Usability %": round(((len(df) - total_missing) / len(df)) * 100, 2)
    })

# 4. Display as a clean DataFrame
quality_report = pd.DataFrame(report_data).set_index("Column")

print(f"--- Data Quality Heartbeat: {current_run_month()} ---")
print(quality_report.to_string())

In [ ]:
df.shape

In [ ]:
import mlflow.sklearn
import pandas as pd

# 1. Use the direct Run URI instead of the Model Name
# This points exactly to the folder we saw in your DagsHub screenshot
RUN_ID = "8f0a412b5b5948118a307b38818ef430"
DIRECT_URI = f"runs:/{RUN_ID}/job_categorizer_model"

print(f"📡 Pulling direct from Run: {RUN_ID}...")

try:
    # 2. Load the pipeline
    loaded_pipeline = mlflow.sklearn.load_model(DIRECT_URI)
    print("✅ Success! Model loaded from direct run path.")

    # 3. Quick Test
    sample_text = ["Senior LLM Engineer with experience in LangChain and PyTorch"]
    prediction = loaded_pipeline.predict(sample_text)
    print(f"🏆 Sample Prediction: {prediction[0]}")

except Exception as e:
    print(f"❌ Direct Load Error: {e}")

In [ ]:
import os
import mlflow

os.environ["MLFLOW_TRACKING_USERNAME"] = "Akashthakur0326"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "d2a8af2664d554a5366508dd80f33e027eb377c2"

mlflow.set_tracking_uri("https://dagshub.com/Akashthakur0326/ResumeRecommender-MLops.mlflow")
